Implementation of Regev's LWE based encryption:

Selection of parameters:
(parameter L indicates the maximal multiplicative depth that the scheme can homomorphically evaluate)

In [43]:
prime=37
dimention=5
number_of_rows=30
L=3

Generation of public keys, homomorphic evaluation key and secret key:

In [44]:
V_1=IntegerModRing(prime)^(dimention)
V_2=IntegerModRing(prime)^(number_of_rows)
V_3=IntegerModRing(prime)^(dimention+1)
T=MatrixSpace(IntegerModRing(prime),number_of_rows,dimention)

secret_keys=[]
for i in range(L+1):
    x=V_1.random_element()
    x=[IntegerModRing(prime)(1)]+list(x)
    secret_keys.append(V_3(x))

l=[]
for i in range(number_of_rows):
    l.append(V_1.random_element())
    
A=T(l)
e=V_2.random_element()
b=A*secret_keys[0][1:]+2*e
print('The public keys:')
print(A)
print('and')
print(b)
print('The secret key:',secret_keys[L-1][1:])


The public keys:
[32 16 28 12 35]
[27 11 11 14 18]
[21  7 12 35  2]
[21 21  8  8 12]
[30 10 26 23 15]
[ 7 26 36 11 18]
[12  9 28 24  8]
[ 1 29  2 29  2]
[ 7  0 29 20 27]
[ 7 20 33  9 28]
[35 33 16  6 34]
[ 3  2 13 23 31]
[ 7  2 35 19 11]
[34 29  9 33  2]
[20  1  3 25 27]
[35 16 33 14 24]
[ 3 16 35 34 11]
[21  2 34 23 25]
[ 4 19 29  6 23]
[ 7 30  3 14 10]
[ 0  2 10 36 18]
[ 7 28 19 31  4]
[12 26 31 20  2]
[ 6 22 28  8 10]
[30 10 36 29 34]
[36 27 13 36  8]
[24 15 21  6 22]
[ 6  9 32 26 18]
[20 14 22 14 35]
[10  9 13 29 21]
and
(13, 18, 17, 10, 7, 14, 33, 34, 1, 5, 25, 16, 31, 26, 19, 19, 5, 25, 32, 5, 28, 16, 36, 0, 24, 20, 35, 12, 2, 35)
The secret key: (13, 32, 22, 18, 3)


Generation of freshly encrypted ciphertexts from input one bit messages

In [47]:
import random
l=[]
for i in range(number_of_rows):
    l.append(random.choice([0,1]))
    
pick=V_2(vector(l))
def fresh_encryption(m):
    v=pick*A
    w=pick*b+IntegerModRing(prime)(m)
    
    return ((v,w),0)
    
m=int(input("Give one bit message:"))
print("The encrypted LWE ciphertext:",fresh_encryption(m))


Give one bit message:0
The encrypted LWE ciphertext: (((3, 9, 22, 21, 21), 2), 0)


In [48]:
v=fresh_encryption(0)[0][0]
w=fresh_encryption(0)[0][1]
w-v*secret_keys[0][1:]

16

In [49]:
def evaluation_decryption(cipher):
    i=cipher[1]
    t=ZZ(cipher[0][1]-cipher[0][0]*secret_keys[i][1:])
    if t%2==0:
        return 0
    else:
        return 1
    
e_vector=list(map(int,input('Give the first component of the cipher:').split(', ')))
e_vector=V_1(vector(e_vector))

s=IntegerModRing(prime)(int(input('Give the second component of the cipher:')))
l=int(input('Give the multiplicative depth of the cipher:'))

print('The message is:', evaluation_decryption(((e_vector,s),l)))   

Give the first component of the cipher:3, 9, 22, 21, 21
Give the second component of the cipher:2
Give the multiplicative depth of the cipher:0
The message is: 0


Generation of homomorphic evaluation key for the somewhat homomorphic evaluation on ciphertexts:

In [50]:
homomorphic_eval_key=[]
for l in range(1,L+1):
    r=[]
    for i in range(0,dimention+1):
        for j in range(i,dimention+1):
            for k in range(0,int(log(prime,2).n(digits=4))+1):
                x=V_1.random_element()
                e=IntegerModRing(prime).random_element()
                y=x*secret_keys[l][1:]+2*e+2**k*secret_keys[l-1][i]*secret_keys[l-1][j]
                r.append((x,y))
                
    homomorphic_eval_key.append(r)
    

Homomorphic addition gate operation on arbitrary number of ciphertexts in the same multiplicative depth 

In [51]:
def homomorphic_addition(l,s):
    v_add=V_1(vector([0 for i in range(dimention)]))
    w_add=IntegerModRing(prime)(0)
    for i in range(len(l)):
        v_add=v_add+l[i][0]
        w_add=w_add+l[i][1]
        
    return ((v_add,w_add),s)

print('First line for each input ciphertext should contain the first component of the ciphertext. Writting stop will stop the process of taking inputs')
ciphertexts=[]
while True:
    try:
        r=V_1(vector(list(map(int,input().split(', ')))))
        s=IntegerModRing(prime)(int(input()))
        ciphertexts.append((r,s))
    except ValueError:
        x=int(input('Give the multiplicative depth of the ciphertexts:'))
        break
print('The added ciphertext:', homomorphic_addition(ciphertexts,x))              

First line for each input ciphertext should contain the first component of the ciphertext. Writting stop will stop the process of taking inputs
11, 6, 7, 8, 91
4
43, 6, 71, 8, 10
30
1, 7, 9, 34, 0
6
stop
Give the multiplicative depth of the ciphertexts:1
The added ciphertext: (((18, 19, 13, 13, 27), 3), 1)


correction check for fresh encrypted ciphertexts
(The message space is GF(2)) :

In [54]:
def fresh_homomorphic_encryption(l):
    c=[]
    for i in range(len(l)):
        c.append(fresh_encryption(l[i]))
        
    h=[c[i][0] for i in range(len(c))]
    return homomorphic_addition(h,0)

l=list(map(int,input().split()))
print('The homomorphic evaluated encryption on the sum of the messages m_1+m_2+...=',sum(l)%2,':',fresh_homomorphic_encryption(l))


1 0 1 1 1
The homomorphic evaluated encryption on the sum of the messages m_1+m_2+...= 0 : (((15, 8, 36, 31, 31), 14), 0)


In [55]:
v=V_1(vector(list(map(int,input().split(', ')))))
w=IntegerModRing(prime)(int(input()))
s=int(input())
print('The message is:', evaluation_decryption(((v,w),0)))

15, 8, 36, 31, 31
14
0
The message is: 0


Homomorphic multiplication gate on two ciphertext inputs in the same multiplicative depth:

In [56]:
def binary_c(n,prime):
    q=n
    l=[]
    while q!=0:
        r=q%2
        q=int(q/2)
        l.append(r)
        
    l.reverse()     
    m=int(log(prime,2).n(digits=4))+1
    q=m-len(l)
    if q>0:
        return [0 for i in range(q)]+l
    else:
        return l

def homomorphic_multiplication(t,l):
    (v_1,v_2)=(t[0][0],t[1][0])
    (w_1,w_2)=(t[0][1],t[1][1])
    v_1=V_3(vector([0]+list(v_1)))
    v_2=V_3(vector([0]+list(v_2)))
    coeff=[]
    for i in range(0,dimention+1):
        for j in range(i,dimention+1):
            if i==0 and j==0:
                x=w_1*w_2
                coeff=coeff+binary_c(ZZ(x),prime)
            elif i==0 and j!=0:
                x=-(w_1*v_2[j]+w_2*v_1[j])
                coeff=coeff+binary_c(ZZ(x),prime)
            elif i!=0:
                if i==j:
                    x=v_1[i]*v_2[i]
                    coeff=coeff+binary_c(ZZ(x),prime)
                elif i!=j:
                    x=v_1[i]*v_2[j]+v_1[j]*v_2[i]
                    coeff=coeff+binary_c(ZZ(x),prime)
                    
    v_mult=V_1(vector([0 for i in range(dimention)]))
    w_mult=IntegerModRing(prime)(0)
    for i in range(len(coeff)):
        v_mult=v_mult+IntegerModRing(prime)(coeff[i])*homomorphic_eval_key[l+1][i][0]
        w_mult=w_mult+IntegerModRing(prime)(coeff[i])*homomorphic_eval_key[l+1][i][1]
        
    return ((v_mult,w_mult),l+1)

print('Give two ciphertext inputs:')
print('First line for each input ciphertext should contain the first component of the ciphertext. Writting stop will stop the process of taking inputs')
ciphertexts=[]
while True:
    try:
        r=V_1(vector(list(map(int,input().split(', ')))))
        s=IntegerModRing(prime)(int(input()))
        ciphertexts.append((r,s))
    except ValueError:
        x=int(input('Give the multiplicative depth of the ciphertexts:'))
        break
print('The multiplied ciphertext:', homomorphic_multiplication(ciphertexts,x))   
   

Give two ciphertext inputs:
First line for each input ciphertext should contain the first component of the ciphertext. Writting stop will stop the process of taking inputs
11, 4, 7, 0, 67
32
6, 0, 98, 7, 8
6
stop
Give the multiplicative depth of the ciphertexts:1
The multiplied ciphertext: (((35, 0, 10, 22, 31), 9), 2)


In [63]:
def evaluation_check(m_1,m_2):
    c_1=fresh_encryption(m_1)
    c_2=fresh_encryption(m_2)
    return homomorphic_multiplication((c_1[0],c_2[0]),0)

m_1=int(input('one bit message:'))
m_2=int(input('one bit message:'))
print('multipled ciphertext of m_1m_2=',m_1*m_2,'is', evaluation_check(m_1,m_2))

one bit message:1
one bit message:0
multipled ciphertext of m_1m_2= 0 is (((25, 10, 23, 17, 8), 1), 1)


In [64]:
e_vector=list(map(int,input('Give the first component of the cipher:').split(', ')))
e_vector=V_1(vector(e_vector))

s=IntegerModRing(prime)(int(input('Give the second component of the cipher:')))
l=int(input('Give the multiplicative depth of the cipher:'))

print('The message is:', evaluation_decryption(((e_vector,s),l)))   

Give the first component of the cipher:25, 10, 23, 17, 8
Give the second component of the cipher:1
Give the multiplicative depth of the cipher:1
The message is: 0


In somewhat homomorphic BV11 scheme we only require to have the decryption of those ciphertexts with multiplicative depth L (the upper bound parameter we set at the beginning ). The ciphertexts with depth L can mainly arise through manipulation on ciphertexts in the intermidiate stages through a function f which is some combination of addition and multiplication gates such that total number of multiplication gates in f is L. The homomorphic evaluation starts from freshly encrypted ciphertexts from input messages. Here one example of evaluations is shown. 

In [67]:
def evaluation(m_1,m_2,m_3,m_4,m_5):
    c_1=fresh_encryption(m_1)
    c_2=fresh_encryption(m_2)
    c_3=fresh_encryption(m_3)
    c_4=fresh_encryption(m_4)
    c_5=fresh_encryption(m_5)
    
    c_new_1=homomorphic_addition([c_1[0],c_2[0]],c_1[1])
    c_new_2=homomorphic_multiplication([c_new_1[0],c_3[0]],c_3[1])
    c_new_3=homomorphic_multiplication([c_4[0],c_5[0]],c_4[1])
    c_new_4=homomorphic_multiplication([c_new_2[0],c_new_3[0]],c_new_2[1])
    
    return c_new_4

l=list(map(int,input('Initial input one bit messages:').split()))
print(evaluation(l[0],l[1],l[2],l[3],l[4]))


Initial input one bit messages:1 0 1 1 1
(((31, 22, 20, 19, 26), 11), 2)


The decryption:

In [68]:
e_vector=list(map(int,input('Give the first component of the cipher:').split(', ')))
e_vector=V_1(vector(e_vector))

s=IntegerModRing(prime)(int(input('Give the second component of the cipher:')))
l=int(input('Give the multiplicative depth of the cipher:'))

print('The message is:', evaluation_decryption(((e_vector,s),l))) 

Give the first component of the cipher:31, 22, 20, 19, 26
Give the second component of the cipher:11
Give the multiplicative depth of the cipher:2
The message is: 1
